#Stage 2   
VIP CUP   
______
**SPA Group**  
Sajjad Jabbarzade   
Parham Soltani  
Amirtaha Aghasi   

______   

##Libraries

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

##Preparing Data   
Importing the road.zip file into the Google VM Drive

In [3]:
from google.colab import files

In [4]:
uploaded = files.upload()

Saving Road.zip to Road.zip


In [5]:
!unzip Road.zip

Archive:  Road.zip
  inflating: Road.mp4                


##Functions

###Lane Detector Algorithm

Canny Edge Detection

In [6]:
def region_selection(image):
	mask = np.zeros_like(image)
	if len(image.shape) > 2:
		channel_count = image.shape[2]
		ignore_mask_color = (255,) * channel_count
	else:
		ignore_mask_color = 255
	rows, cols = image.shape[:2]
	bottom_left = [cols * 0.1, rows * 0.95]
	top_left	 = [cols * 0.4, rows * 0.6]
	bottom_right = [cols * 0.9, rows * 0.95]
	top_right = [cols * 0.6, rows * 0.6]
	vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
	cv2.fillPoly(mask, vertices, ignore_mask_color)
	masked_image = cv2.bitwise_and(image, mask)
	return masked_image

Extract the region of interest in the input image with Hough Transform

In [7]:
def hough_transform(image):
	rho = 1 # Distance resolution of the accumulator in pixels
	theta = np.pi/180 # Angle resolution of the accumulator in radians
	threshold = 20
	minLineLength = 20
	maxLineGap = 500
	return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold,
						minLineLength = minLineLength, maxLineGap = maxLineGap)

###Plot Lines on video frames

1. Average_Slope_Intercept: Find the slope and intercept of the left and right lanes of each image

In [8]:
def average_slope_intercept(lines):
	left_lines = [] #(slope, intercept)
	left_weights = [] #(length,)
	right_lines = [] #(slope, intercept)
	right_weights = [] #(length,)
	for line in lines:
		for x1, y1, x2, y2 in line:
			if x1 == x2:
				continue
			slope = (y2 - y1) / (x2 - x1)
			intercept = y1 - (slope * x1)
			length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
			if slope < 0:
				left_lines.append((slope, intercept))
				left_weights.append((length))
			else:
				right_lines.append((slope, intercept))
				right_weights.append((length))
	left_lane = np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) > 0 else None
	right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
	return left_lane, right_lane

2. Pixel_Points: Converts the slope and intercept of each line into pixel points

In [9]:
def pixel_points(y1, y2, line):
	if line is None:
		return None
	slope, intercept = line
	x1 = int((y1 - intercept)/slope)
	x2 = int((y2 - intercept)/slope)
	y1 = int(y1)
	y2 = int(y2)
	return ((x1, y1), (x2, y2))

3. Lane_Lines: Create full lenght lines from pixel points

In [10]:
def lane_lines(image, lines):
	left_lane, right_lane = average_slope_intercept(lines)
	y1 = image.shape[0]
	y2 = y1 * 0.6
	left_line = pixel_points(y1, y2, left_lane)
	right_line = pixel_points(y1, y2, right_lane)
	return left_line, right_line

4. Draw_Lane_Lines: Draw lines onto the input image

In [11]:
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=12):
	line_image = np.zeros_like(image)
	for line in lines:
		if line is not None:
			cv2.line(line_image, *line, color, thickness)
	return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

###Processing

In [12]:
def frame_processor(image):
	grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	kernel_size = 5
	blur = cv2.GaussianBlur(grayscale, (kernel_size, kernel_size), 0)
	low_t = 50
	high_t = 150
	edges = cv2.Canny(blur, low_t, high_t)
	region = region_selection(edges)
	hough = hough_transform(region)
	result = draw_lane_lines(image, lane_lines(image, hough))
	return result

In [13]:
def get_fps(inputvideo):
  return int(inputvideo.get(cv2.CAP_PROP_FPS))

In [17]:
def process_video(inputvideopath, outputvideopath):
  input_video = cv2.VideoCapture(inputvideopath)
  width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = get_fps(input_video)
  print(f"The FPS of the input video = {fps}")
  output_video = cv2.VideoWriter(outputvideopath, cv2.VideoWriter_fourcc(*"XVID"), fps, (width, height))
  while True:
      ok, frame = input_video.read()
      if not ok:
          break
      processed_frame = frame_processor(frame)
      output_video.write(processed_frame)
  input_video.release()
  output_video.release()
  print("|---------- Done ----------|")

##Task 3

In [15]:
video_input_file_name = "/content/Road.mp4"
video_output_file_name = "Our Output.mp4"

In [16]:
process_video(video_input_file_name,video_output_file_name)

50
